### Read the GDP release with Python

In [1]:
import requests
import pandas as pd
import config   ## File with API key

def write_txt(filename, filetext):
    '''Write label to txt file'''
    with open(filename, 'w') as text_file:
        text_file.write(filetext)

In [2]:
# Components of request
base = f'https://www.bea.gov/api/data/?&UserID={config.bea_key}'
dset = '&method=GetData&datasetname=NIPA'
tbl = [('T10102', ['A191RL', 'DPCERY', 'A006RY', 'A019RY', 'A822RY']),
       ('T50302', ['A008RZ', 'B009RZ', 'Y033RZ', 'Y001RZ']),
       ('T20306', ['DPCERX', 'DDURRX', 'DNDGRX', 'DSERRX', 'DHUTRX'])]
       #('T50100', ['W170RC', 'W988RC', 'W987RC', 'A782RC']),
       #('T10106', ['A191RX', 'RPCERX', 'A005RX', 'A019RX', 'A822RX'])]
freq = '&Frequency=Q'
year = '&Year=ALL' # {",".join([str(i) for i in range(2005,2018)])}
fmt = '&ResultFormat=json'

s1,s2,s3,s4 = '\scriptsize{','}\\\\\scriptsize{','}\\\\\\textbf{','}'

In [3]:
for t in tbl:
    # Build url from components
    url = f'{base}{dset}&TableName={t[0]}{freq}{year}{fmt}'
    # Request data
    full_table = requests.get(url).json()['BEAAPI']['Results']['Data']
    r = [s for s in full_table if s['SeriesCode'] in t[1]]
    idx1 = [pd.to_datetime(i['TimePeriod']) for i in r]
    idx2 = [i['SeriesCode'] for i in r]
    df = pd.DataFrame(r, index=[idx1, idx2], columns=['DataValue']).unstack(1)['DataValue']
    
    # Break out PCE
    if t[0] == 'T20306':
        df = pd.DataFrame(
            [df[col].str.replace(',','').astype(float) 
             for col in df]).T
        df['DSEREXHUT'] = df['DSERRX'] - df['DHUTRX']
        dft = df.diff()
        dft = dft.div(dft['DPCERX'], axis=0)
        df = dft.multiply((((df['DPCERX'].pct_change() + 1) ** 4) - 1) * 100, axis=0)
    
    df.loc['2000-01-01':].to_csv(f'{t[0]}.csv', index_label='DATE')

    # Write chart label to txt
    lt_dt = r[-1]['TimePeriod']
    lt_val = round(float(df[t[1][0]].iloc[-1:][0]), 1)
    lbl = f'{s1}{lt_dt[:4]}{s2}{lt_dt[4:]}{s3}{lt_val}{s4}'
    write_txt(f'{t[0]}.txt', lbl)

In [4]:
df

,DDURRX,DHUTRX,DNDGRX,DPCERX,DSERRX,DSEREXHUT
1999-01-01,NaN,NaN,NaN,NaN,NaN,NaN
1999-04-01,1.704514,0.798550,0.888303,6.057136,2.794709,1.996159
1999-07-01,0.877552,0.849135,0.437513,4.617460,3.091322,2.242188
1999-10-01,0.446329,0.007269,1.922014,6.007065,3.725163,3.717895
2000-01-01,2.075109,0.421025,-0.832097,6.226154,4.298091,3.877066
2000-04-01,-0.708735,1.087688,1.831184,3.896078,3.351428,2.263740
2000-07-01,0.633350,0.726431,0.611689,3.946933,2.592342,1.865912
2000-10-01,0.259775,1.292308,0.890304,3.609992,2.500987,1.208679
2001-01-01,0.611462,0.412484,-0.549023,1.700810,1.442503,1.030019
2001-04-01,-0.032676,-0.081521,0.423871,1.030862,0.683927,0.765448


In [41]:
df.iloc[-5]

DDURRX       1.252155
DHUTRX      -0.159543
DNDGRX       0.532019
DPCERX       2.924330
DSERRX       1.353680
DSEREXHUT    1.513223
Name: 2016-10-01 00:00:00, dtype: float64

In [22]:
r[-1]['TimePeriod']

'2017Q4'

In [24]:
df['DSEREXHUT'] = df['DSERRX'] - df['DHUTRX']
dft = df.diff()
dft = dft.div(dft['DPCERX'], axis=0)

In [35]:
dft.multiply((((df['DPCERX'].pct_change() + 1) ** 4) - 1) * 100, axis=0)

,DDURRX,DHUTRX,DNDGRX,DPCERX,DSERRX,DSEREXHUT
1999-01-01,NaN,NaN,NaN,NaN,NaN,NaN
1999-04-01,1.704514,0.798550,0.888303,6.057136,2.794709,1.996159
1999-07-01,0.877552,0.849135,0.437513,4.617460,3.091322,2.242188
1999-10-01,0.446329,0.007269,1.922014,6.007065,3.725163,3.717895
2000-01-01,2.075109,0.421025,-0.832097,6.226154,4.298091,3.877066
2000-04-01,-0.708735,1.087688,1.831184,3.896078,3.351428,2.263740
2000-07-01,0.633350,0.726431,0.611689,3.946933,2.592342,1.865912
2000-10-01,0.259775,1.292308,0.890304,3.609992,2.500987,1.208679
2001-01-01,0.611462,0.412484,-0.549023,1.700810,1.442503,1.030019
2001-04-01,-0.032676,-0.081521,0.423871,1.030862,0.683927,0.765448


In [16]:
dft = df['DataValue'].diff()

In [34]:
for col in df['DataValue'].keys():
    df['DataValue'][col] = df['DataValue'][col].str.replace(',','').astype(float)

In [42]:
df['DataValue'] = pd.DataFrame([df['DataValue'][col].str.replace(',','').astype(float) for col in df['DataValue'].keys()]).T

In [31]:
df['DataValue'][col].str.replace(',','').astype(float)

1999-01-01    5245389.0
1999-04-01    5297451.0
1999-07-01    5356195.0
1999-10-01    5427430.0
2000-01-01    5510763.0
2000-04-01    5577287.0
2000-07-01    5629228.0
2000-10-01    5679888.0
2001-01-01    5709575.0
2001-04-01    5723745.0
2001-07-01    5733702.0
2001-10-01    5756818.0
2002-01-01    5791933.0
2002-04-01    5825165.0
2002-07-01    5848159.0
2002-10-01    5887428.0
2003-01-01    5909399.0
2003-04-01    5942311.0
2003-07-01    5986872.0
2003-10-01    6028993.0
2004-01-01    6083984.0
2004-04-01    6126762.0
2004-07-01    6177086.0
2004-10-01    6238426.0
2005-01-01    6271489.0
2005-04-01    6331814.0
2005-07-01    6388449.0
2005-10-01    6421912.0
2006-01-01    6464403.0
2006-04-01    6511278.0
                ...    
2010-07-01    6743979.0
2010-10-01    6785643.0
2011-01-01    6811987.0
2011-04-01    6839240.0
2011-07-01    6876640.0
2011-10-01    6877739.0
2012-01-01    6898047.0
2012-04-01    6905967.0
2012-07-01    6909712.0
2012-10-01    6918754.0
2013-01-01    69

In [65]:
lt_dt = r[-1]['TimePeriod']
lt_val = round(float(df['DataValue']['A191RL'].iloc[-1:][0]), 2)

In [61]:
lt_dt

Timestamp('2017-10-01 00:00:00')

In [15]:
#df = df['DataValue']
df['DNHUSSER'] = df['DSERRX'].str.replace(',','').astype(float) - df['DHUTRX'].str.replace(',','').astype(float)

In [16]:
df

,DDURRX,DHUTRX,DNDGRX,DPCERX,DSERRX,DNHUSSER
1999-01-01,"664,025","1,531,197","1,780,218","7,618,691","5,245,389",3714192.0
1999-04-01,"695,778","1,546,073","1,796,766","7,731,528","5,297,451",3751378.0
1999-07-01,"712,454","1,562,209","1,805,080","7,819,273","5,356,195",3793986.0
1999-10-01,"720,989","1,562,348","1,841,834","7,934,144","5,427,430",3865082.0
2000-01-01,"761,222","1,570,511","1,825,701","8,054,859","5,510,763",3940252.0
2000-04-01,"747,154","1,592,101","1,862,049","8,132,194","5,577,287",3985186.0
2000-07-01,"759,844","1,606,656","1,874,305","8,211,276","5,629,228",4022572.0
2000-10-01,"765,106","1,632,833","1,892,339","8,284,400","5,679,888",4047055.0
2001-01-01,"777,690","1,641,322","1,881,040","8,319,403","5,709,575",4068253.0
2001-04-01,"777,013","1,639,633","1,889,822","8,340,761","5,723,745",4084112.0


In [ ]:
f'{df.iloc[-1].name: %Y %}'#df.iloc[-1].name

In [ ]:
lt_date = r[-1]['TimePeriod']

s1, s2, s3, s4 = '\scriptsize{', '}\\\scriptsize{', '}\\\textbf{', '}'
lbl = f'{s1}{lt_date[:4]}{s2}{lt_date[4:]}{s3}{df["A191RL"][-1]}{s4}'

In [ ]:
lt_date[4:]

In [ ]:
lbl = f'{s1}{lt_date[:4]}{s2}{lt_date[4:]}{s3}{df["A191RL"][-1]}{s4}'

In [5]:
# Build url from components
url = f'{base}{dset}&TableName={tbl[0][0]}{freq}{year}{fmt}'

# Request data and and convert to pandas dataframe
r = requests.get(url).json()['BEAAPI']['Results']['Data']
dates = [pd.to_datetime(v['TimePeriod']) for v in r]
df = pd.DataFrame(r, index=[dates, 'SeriesCode'])
df = df[df['SeriesCode'].isin(tbl[0][1])].set_index(['date', 'SeriesCode'])
df = df['DataValue'].unstack(1).loc['2006-01-01':]
df.to_csv('gdp_comp.csv', index_label='DATE')

write_txt('gdp_comp.txt', df['A191RL'][-1]) # Write chart label to txt

ValueError: Shape of passed values is (9, 7384), indices imply (9, 2)

In [ ]:
# Build url from components
url = f'{base}{dset}&TableName={tbl[1][0]}{freq}{year}{fmt}'

# Request data and and convert to pandas dataframe
r = requests.get(url).json()['BEAAPI']['Results']['Data']
df = pd.DataFrame(r, index=[pd.to_datetime(v['TimePeriod']) for v in r])

In [ ]:
# Build url from components
url = f'{base}{dset}&TableName={tbl[3][0]}{freq}{year}{fmt}'

# Request data and and convert to pandas dataframe
r = requests.get(url).json()['BEAAPI']['Results']['Data']
df = pd.DataFrame(r, index=[pd.to_datetime(v['TimePeriod']) for v in r])

In [ ]:
df['2017-10-01']#['LineDescription'].unique()

In [ ]:
r = [x for x in r if x['SeriesCode'] in tbl[0][1]]
idx1 = [pd.to_datetime(i['TimePeriod']) for i in r]
idx2 = [i['SeriesCode'] for i in r]
df = pd.DataFrame(r, index=[idx1, idx2])['DataValue'].unstack(1)

In [ ]:
pd.DataFrame(r, index=[idx1, idx2])['DataValue'].unstack(1)

In [ ]:
df

In [ ]:
com = 'T10102'
inv = 'T50302'
gdi = 'T50100'

In [ ]:
url = '{}&datasetname={}&TableName={}&Frequency={}&Year={}&ResultFormat=json'.format(
    base, 'NIPA', 'T10105', 'Q', 'ALL')    # Adjust here as needed
df = pd.DataFrame(requests.get(url).json()['BEAAPI']['Results']['Data'])

In [ ]:
float(Y.replace(',', ''))

In [ ]:
(df[df['SeriesCode'] == 'A191RC'].set_index('TimePeriod')['DataValue'].str.replace(',', '').astype('float') * 1000).loc['2006-01-01':]

In [ ]:
[x for x in r if x['SeriesCode'] in tbl[0][1]]

In [ ]:
tbl[0][1]

In [ ]:
r

In [ ]:
import requests
import config   ## File with API key
bea_key = config.bea_key

# Components of request
base = 'https://www.bea.gov/api/data/?&UserID={}'.format(bea_key)
m = '&method=GetData'
dsn = '&datasetname=NIPA'
ind = '&TableId=2'
freq = '&Frequency=Q'
year = '&Year=LAST12'
fmt = '&ResultFormat=json'

# Combined url for request
url = '{}{}{}{}{}{}{}'.format(base, m, dsn, ind, freq, year, fmt)
r = requests.get(url).json()
#f = pd.DataFrame(r['BEAAPI']['Results']['Data'])

In [ ]:
r